In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [2]:
HPCVersion = False # the only thing that changes is the testfolder/ewpfile. The other are for speed.

if HPCVersion:
    testfolder = '/scratch/sayala/RadianceScenes/Test1'
    epwfile='/scratch/sayala/EPWs/USA_VA_Richmond.Intl.AP.724010_TMY.epw'
    nMods = 20
    nRows = 7
    sensorsy = 210  # this will give 70 sensors per module.
    modWanted = nMods/2
    rowWanted = nRows/2
else: # Run Silvana computer version faster
    testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo3'
    epwfile = r'C:\Users\sayala\Documents\RadianceScenes\Demo3\EPWs\USA_VA_Richmond.Intl.AP.724010_TMY.epw'
    nMods = 1  # replicating the unit 3 times
    nRows = 1  # only 1 row
    rowWanted = 1
    modWanted = 1
    sensorsy = 9

In [3]:
simulationname = 'TestHPC1'
moduletype = 'Silvana'
daydate = '02_24'

####

# Control Variables
cumulativesky=False
rewriteModule=True
hpc=True

# Scene Variables 
gcr = 0.35   # ground cover ratio,  = module_height / pitch
albedo = 0.3     # ground albedo

# Fixed tilt variables
azimuth_ang=180 # Facing south
tilt =20 # tilt. 

# Tracker geometry options
backtrack = True
hub_height = 2   # tracker height at 0 tilt in meters (hub height)
limit_angle = 45 # tracker rotation limit angle
axis_azimuth=180
angledelta=5

# ModuleDict Variables
numpanels = 1
x=0.984  
y = 1.996
bifi=0.9
xgap = 0.01 # 10 centimeters.
ygap = 0.0
zgap = 0.1

# TorqueTube Parameters
torqueTube = True
axisofrotationTorqueTube=False
diameter=0.1
tubetype='Round'
torqueTubeMaterial='Metal_Grey'

# Cell Level Module Parameters
cellLevelModule = False
numcellsx=10
numcellsy=6
xcell=0.156
ycell=0.156
xcellgap=0.02
ycellgap=0.02


In [4]:
demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=bifi, 
                       torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                       material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                       rewriteModulefile = True, xgap=xgap, 
                       axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                       numcellsx=numcellsx, numcellsy = numcellsy)
sceneDict = {'module_type':moduletype, 'pitch': round(moduleDict['sceney'] / gcr,3),'height':hub_height, 'nMods':nMods, 'nRows':nRows} #, 'nMods':20, 'nRows':7}  

#epwfile = demo.getEPW(35.0375,-106.6055) # pull EPW data for CO Boulder
metdata = demo.readEPW(epwfile=epwfile, hpc=True, daydate=daydate) # read in the EPW weather data from above
trackerdict = demo.set1axis(cumulativesky=cumulativesky, axis_azimuth=axis_azimuth, limit_angle=limit_angle, angledelta=angledelta, backtrack=backtrack, gcr=gcr)
trackerdict = demo.gendaylit1axis(hpc=hpc)  #benchmark time: gendaylit2:105s.  gendaylit: 5s



path = C:\Users\sayala\Documents\RadianceScenes\Demo3
('\nModule Name:', 'Silvana')
REWRITING pre-existing module file. 
Module Silvana successfully created
Creating ~4000 skyfiles.  Takes 1-2 minutes
Created 12 skyfiles in /skies/


C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:424: RuntimeWarning: invalid value encountered in minimum
  temp = np.minimum(axes_distance*cosd(wid), 1)
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:431: RuntimeWarning: invalid value encountered in less
  tracker_theta = np.where(wid < 0, wid + wc, wid - wc)
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:435: RuntimeWarning: invalid value encountered in greater
  tracker_theta[tracker_theta > max_angle] = max_angle
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:436: RuntimeWarning: invalid value encountered in less
  tracker_theta[tracker_theta < -max_angle] = -max_angle
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:543: RuntimeWarning: invalid value encountered in less
  surface_azimuth[surface_azimuth < 0] += 360
C:\ProgramData\Anaconda2\lib\site-packages\pvlib\tracking.py:544: RuntimeWarning: invalid value encountered in greater_equal
  surface_azimuth[surface_a

In [5]:
# Sanity Check of SUnrise and Sunset Hours all Year:
#metdata = demo.readEPW(epwfile=epwfile) # read in the EPW weather data from above
#trackerdict = demo.set1axis(cumulativesky=cumulativesky, axis_azimuth=axis_azimuth, limit_angle=limit_angle, angledelta=angledelta, backtrack=backtrack, gcr=gcr)
#trackerdict = demo.gendaylit1axis()  #benchmark time: gendaylit2:105s.  gendaylit: 5s

In [6]:
trackerdict = demo.makeScene1axis(trackerdict=trackerdict, moduletype=moduletype, sceneDict=sceneDict, cumulativesky=cumulativesky) #makeScene creates a .rad file with 20 modules per row, 7 rows.
trackerdict = demo.makeOct1axis(trackerdict=trackerdict, hpc=True)


Making ~4000 .rad files for gendaylit 1-axis workflow (this takes a minute..)
12 Radfiles created in /objects/

Making 12 octfiles for 1-axis tracking in root directory.
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_24_10.rad
HPC Checking for file objects\1axis02_24_10_1.294_5.703_1x1.rad
Created 1axis_02_24_10.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_24_08.rad
HPC Checking for file objects\1axis02_24_08_1.750_5.703_1x1.rad
Created 1axis_02_24_08.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_24_09.rad
HPC Checking for file objects\1axis02_24_09_1.294_5.703_1x1.rad
Created 1axis_02_24_09.oct
HPC Checking for file materials\ground.rad
Missing files, skipping...
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_24_15.rad
HPC Checking for file objects\1axis02_24_15_1.323_5.703_1x1.rad
Created 1axis_02_2

In [7]:
trackerdict = demo.analysis1axis(trackerdict, modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)


(1.9905401540586054, 90.0, 0.5431035711385306, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_24_07_Front
Linescan in process: 1axis_02_24_07_Back
Saved: results\irr_1axis_02_24_07.csv
Index: 02_24_07. Wm2Front: 2.960287. Wm2Back: 0.804592959259
(1.7502848918769343, 90.0, 14.490267047481026, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_24_08_Front
Linescan in process: 1axis_02_24_08_Back
Saved: results\irr_1axis_02_24_08.csv
Index: 02_24_08. Wm2Front: 285.478166667. Wm2Back: 51.0645514815
(1.2943074323758257, 90.0, 44.99999999999999, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_24_09_Front
Linescan in process: 1axis_02_24_09_Back
Saved: results\irr_1axis_02_24_09.csv
Index: 02_24_09. Wm2Front: 735.700755556. Wm2Back: 107.311968889
(1.2943074323758257, 90.0, 44.99999999999999, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_24_10_Front
Linescan in process: 1axis_02_24_10_Back
Saved: results\irr_1axis_02_24_10.csv
Index: 02_24_10. Wm

In [33]:
#epwfile = demo.getEPW(35.0375,-106.6055) # pull EPW data for CO Boulder
# loop to see if it crashes

daydateall=['02_16',
        '02_17',
         '02_18',
         '02_19',
         '02_20',
         '02_21',
         '02_22',
         '02_23',
         '02_24',
         '02_25',
         '02_26',
         '02_27',
         '02_28',
         '03_01',
         '03_02']



for i in range (0,len(daydateall)):
    daydate = daydateall[i]
    metdata = demo.readEPW(epwfile=epwfile, hpc=True, daydate=daydate) # read in the EPW weather data from above
    trackerdict = demo.set1axis(cumulativesky=cumulativesky, axis_azimuth=axis_azimuth, limit_angle=limit_angle, angledelta=angledelta, backtrack=backtrack, gcr=gcr)
    trackerdict = demo.gendaylit1axis(hpc=hpc)  #benchmark time: gendaylit2:105s.  gendaylit: 5s
    trackerdict = demo.makeScene1axis(trackerdict=trackerdict, moduletype=moduletype, sceneDict=sceneDict, cumulativesky=cumulativesky) #makeScene creates a .rad file with 20 modules per row, 7 rows.
    trackerdict = demo.makeOct1axis(trackerdict=trackerdict, hpc=True)
    trackerdict = demo.analysis1axis(trackerdict, modWanted=modWanted, rowWanted=rowWanted, sensorsy=sensorsy)



Creating ~4000 skyfiles.  Takes 1-2 minutes
Created 10 skyfiles in /skies/

Making ~4000 .rad files for gendaylit 1-axis workflow (this takes a minute..)
10 Radfiles created in /objects/

Making 10 octfiles for 1-axis tracking in root directory.
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_16_14.rad
HPC Checking for file objects\1axis02_16_14_1.585_5.703_1x1.rad
Created 1axis_02_16_14.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_16_15.rad
HPC Checking for file objects\1axis02_16_15_1.306_5.703_1x1.rad
Created 1axis_02_16_15.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_16_16.rad
HPC Checking for file objects\1axis02_16_16_1.294_5.703_1x1.rad
Created 1axis_02_16_16.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_16_17.rad
HPC Checking for file objects\1axis02_16_17_1.438_5.703_1x1.rad
Created 1axis

Linescan in process: 1axis_02_17_13_Back
Saved: results\irr_1axis_02_17_13.csv
Index: 02_17_13. Wm2Front: 564.932911111. Wm2Back: 156.069118519
(1.5872000133945698, 90.0, -24.432897023621223, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_17_14_Front
Linescan in process: 1axis_02_17_14_Back
Saved: results\irr_1axis_02_17_14.csv
Index: 02_17_14. Wm2Front: 471.848233333. Wm2Back: 120.155971111
(1.3084589403286138, 90.0, -43.862248354080236, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_17_15_Front
Linescan in process: 1axis_02_17_15_Back
Saved: results\irr_1axis_02_17_15.csv
Index: 02_17_15. Wm2Front: 353.218833333. Wm2Back: 88.6060248148
(1.2943074323758257, 90.0, -44.99999999999999, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_17_16_Front
Linescan in process: 1axis_02_17_16_Back
Saved: results\irr_1axis_02_17_16.csv
Index: 02_17_16. Wm2Front: 212.909077778. Wm2Back: 52.6464422222
(1.427734586460669, 90.0, -34.98851591516699, 1, 1, 5.703, 0, 1.9

Linescan in process: 1axis_02_19_08_Back
Saved: results\irr_1axis_02_19_08.csv
Index: 02_19_08. Wm2Front: 141.706677778. Wm2Back: 31.3293659259
(1.2943074323758257, 90.0, 44.99999999999999, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_19_09_Front
Linescan in process: 1axis_02_19_09_Back
Saved: results\irr_1axis_02_19_09.csv
Index: 02_19_09. Wm2Front: 417.050622222. Wm2Back: 75.586662963
(1.2943074323758257, 90.0, 44.99999999999999, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_19_10_Front
Linescan in process: 1axis_02_19_10_Back
Saved: results\irr_1axis_02_19_10.csv
Index: 02_19_10. Wm2Front: 542.412522222. Wm2Back: 108.928903333
(1.365648793824352, 90.0, 39.46616580220379, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_19_11_Front
Linescan in process: 1axis_02_19_11_Back
Saved: results\irr_1axis_02_19_11.csv
Index: 02_19_11. Wm2Front: 603.2282. Wm2Back: 135.868125926
(1.6666293699957069, 90.0, 19.514092845813863, 1, 1, 5.703, 0, 1.996, 0.994)


Created 1axis_02_21_08.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_21_14.rad
HPC Checking for file objects\1axis02_21_14_1.593_5.703_1x1.rad
Created 1axis_02_21_14.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_21_15.rad
HPC Checking for file objects\1axis02_21_15_1.317_5.703_1x1.rad
Created 1axis_02_21_15.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_21_12.rad
HPC Checking for file objects\1axis02_21_12_1.671_5.703_1x1.rad
Created 1axis_02_21_12.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_21_13.rad
HPC Checking for file objects\1axis02_21_13_1.955_5.703_1x1.rad
Created 1axis_02_21_13.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_21_10.rad
HPC Checking for file objects\1axis02_21_10_1.294_5.703_1x1.rad
Created 1axis_02_21_10.oct
HPC Checking 

Saved: results\irr_1axis_02_22_16.csv
Index: 02_22_16. Wm2Front: 44.7400088889. Wm2Back: 17.2279340741
(1.374283398458303, 90.0, -38.82696664568667, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_22_17_Front
Linescan in process: 1axis_02_22_17_Back
Saved: results\irr_1axis_02_22_17.csv
Index: 02_22_17. Wm2Front: 21.1766377778. Wm2Back: 7.37448740741
Creating ~4000 skyfiles.  Takes 1-2 minutes
Created 10 skyfiles in /skies/

Making ~4000 .rad files for gendaylit 1-axis workflow (this takes a minute..)
10 Radfiles created in /objects/

Making 10 octfiles for 1-axis tracking in root directory.
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_23_14.rad
HPC Checking for file objects\1axis02_23_14_1.596_5.703_1x1.rad
Created 1axis_02_23_14.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_23_15.rad
HPC Checking for file objects\1axis02_23_15_1.321_5.703_1x1.rad
Created 1axis_02_23_15.oct
HPC C

Linescan in process: 1axis_02_24_09_Back
Saved: results\irr_1axis_02_24_09.csv
Index: 02_24_09. Wm2Front: 735.775988889. Wm2Back: 107.536953704
(1.2943074323758257, 90.0, 44.99999999999999, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_24_10_Front
Linescan in process: 1axis_02_24_10_Back
Saved: results\irr_1axis_02_24_10.csv
Index: 02_24_10. Wm2Front: 852.752777778. Wm2Back: 147.606733333
(1.3810100389920303, 90.0, 38.33296305078301, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_24_11_Front
Linescan in process: 1axis_02_24_11_Back
Saved: results\irr_1axis_02_24_11.csv
Index: 02_24_11. Wm2Front: 758.478033333. Wm2Back: 164.064107407
(1.678894185425391, 90.0, 18.76874896854271, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_24_12_Front
Linescan in process: 1axis_02_24_12_Back
Saved: results\irr_1axis_02_24_12.csv
Index: 02_24_12. Wm2Front: 632.108433333. Wm2Back: 164.590318519
(1.9537883964631397, 90.0, -2.653984888479556, 1, 1, 5.703, 0, 1.996, 0

Created 1axis_02_26_11.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_26_10.rad
HPC Checking for file objects\1axis02_26_10_1.294_5.703_1x1.rad
Created 1axis_02_26_10.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_26_17.rad
HPC Checking for file objects\1axis02_26_17_1.329_5.703_1x1.rad
Created 1axis_02_26_17.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_26_16.rad
HPC Checking for file objects\1axis02_26_16_1.294_5.703_1x1.rad
Created 1axis_02_26_16.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_26_15.rad
HPC Checking for file objects\1axis02_26_15_1.327_5.703_1x1.rad
Created 1axis_02_26_15.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_02_26_07.rad
HPC Checking for file objects\1axis02_26_07_1.979_5.703_1x1.rad
Created 1axis_02_26_07.oct
HPC Checking 

Linescan in process: 1axis_02_27_11_Back
Saved: results\irr_1axis_02_27_11.csv
Index: 02_27_11. Wm2Front: 466.723311111. Wm2Back: 118.10832037
(1.6867461999906856, 90.0, 18.293307986044724, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_27_12_Front
Linescan in process: 1axis_02_27_12_Back
Saved: results\irr_1axis_02_27_12.csv
Index: 02_27_12. Wm2Front: 544.455966667. Wm2Back: 143.0433
(1.9516175622680847, 90.0, -2.7787540018834704, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_27_13_Front
Linescan in process: 1axis_02_27_13_Back
Saved: results\irr_1axis_02_27_13.csv
Index: 02_27_13. Wm2Front: 592.590477778. Wm2Back: 162.696140741
(1.6012473928859177, 90.0, -23.55015673220062, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_02_27_14_Front
Linescan in process: 1axis_02_27_14_Back
Saved: results\irr_1axis_02_27_14.csv
Index: 02_27_14. Wm2Front: 497.832266667. Wm2Back: 126.744174074
(1.3295130899391414, 90.0, -42.208506861082334, 1, 1, 5.703, 0, 1.996, 0

Created 1axis_03_01_14.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_03_01_17.rad
HPC Checking for file objects\1axis03_01_17_1.294_5.703_1x1.rad
Created 1axis_03_01_17.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_03_01_16.rad
HPC Checking for file objects\1axis03_01_16_1.294_5.703_1x1.rad
Created 1axis_03_01_16.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_03_01_11.rad
HPC Checking for file objects\1axis03_01_11_1.399_5.703_1x1.rad
Created 1axis_03_01_11.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_03_01_07.rad
HPC Checking for file objects\1axis03_01_07_1.966_5.703_1x1.rad
Created 1axis_03_01_07.oct
HPC Checking for file materials\ground.rad
HPC Checking for file skies\sky2_37.5_-77.33_03_01_13.rad
HPC Checking for file objects\1axis03_01_13_1.949_5.703_1x1.rad
Created 1axis_03_01_13.oct
HPC Checking 

Linescan in process: 1axis_03_02_09_Back
Saved: results\irr_1axis_03_02_09.csv
Index: 03_02_09. Wm2Front: 652.765455556. Wm2Back: 104.768445185
(1.2943074323758257, 90.0, 44.99999999999999, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_03_02_10_Front
Linescan in process: 1axis_03_02_10_Back
Saved: results\irr_1axis_03_02_10.csv
Index: 03_02_10. Wm2Front: 751.327874074. Wm2Back: 140.026577778
(1.4032657001429985, 90.0, 36.721799674429825, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_03_02_11_Front
Linescan in process: 1axis_03_02_11_Back
Saved: results\irr_1axis_03_02_11.csv
Index: 03_02_11. Wm2Front: 735.399544444. Wm2Back: 163.176611111
(1.6970591583037358, 90.0, 17.67082566657693, 1, 1, 5.703, 0, 1.996, 0.994)
Linescan in process: 1axis_03_02_12_Front
Linescan in process: 1axis_03_02_12_Back
Saved: results\irr_1axis_03_02_12.csv
Index: 03_02_12. Wm2Front: 672.588355556. Wm2Back: 175.005692593
(1.948227673264114, 90.0, -2.97361512625848, 1, 1, 5.703, 0, 1.996, 0

# SOL POS Sanity Check

In [26]:
import pandas as pd
import pytz
import pvlib

lat = metdata.latitude
lon=metdata.longitude
elev=metdata.elevation
datetime_original=metdata.datetime[11]
datetime = pd.to_datetime(datetime_original)
tz = metdata.timezone
try:  # make sure the data is tz-localized.
    datetimetz = datetime.tz_localize(pytz.FixedOffset(tz*60))  # either use pytz.FixedOffset (in minutes) or 'Etc/GMT+5'
except:  # data is tz-localized already. Just put it in local time.
    datetimetz = datetime.tz_convert(pytz.FixedOffset(tz*60))  

print ("Metdata timestamp metdata.datetime[11]= %s " % datetime_original)
print ("Datetime pd. %s " % datetime)
print ("Datetime localized %s " % datetimetz)

solpos = pvlib.irradiance.solarposition.get_solarposition(datetimetz,lat,lon,elev)
solpos = pvlib.irradiance.solarposition.get_solarposition(datetimetz-pd.Timedelta(minutes = 30),lat,lon,elev)

print ("\n Solposition: %s" % solpos)

axis_tilt=0
axis_azimuth=180
limit_angle=45
backtrack=True

trackingdata = pvlib.tracking.singleaxis(solpos['zenith'], solpos['azimuth'], axis_tilt, axis_azimuth, limit_angle, backtrack, gcr)
tracker_theta = trackingdata['tracker_theta'].tolist()

print ("\nTracker theta %s " % tracker_theta)
print ("\n Full tracking data %s " % trackingdata)
print ("\nQuerying pandas datetime.hour %s " % datetime.hour)
    

Metdata timestamp metdata.datetime[11]= 1971-02-24 18:00:00-05:00 
Datetime pd. 1971-02-24 18:00:00-05:00 
Datetime localized 1971-02-24 18:00:00-05:00 

 Solposition:                            apparent_elevation  apparent_zenith     azimuth  \
1971-02-24 17:30:00-05:00            4.664168        85.335832  254.480729   

                           elevation  equation_of_time     zenith  
1971-02-24 17:30:00-05:00   4.491443        -13.304559  85.508557  

Tracker theta [8.76330741432561] 

 Full tracking data                            tracker_theta        aoi  surface_azimuth  \
1971-02-24 17:30:00-05:00       8.763307  77.071001            270.0   

                           surface_tilt  
1971-02-24 17:30:00-05:00      8.763307   

Querying datetime.hour 18 
